# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd164332c70>
├── 'a' --> tensor([[-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961]])
└── 'x' --> <FastTreeValue 0x7fd164332ca0>
    └── 'c' --> tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                        [-0.4802, -0.7260, -0.2455, -1.1421],
                        [ 0.3915,  0.8803, -0.6810, -0.8067]])

In [4]:
t.a

tensor([[-0.0387,  2.1786,  1.0509],
        [ 1.2989,  0.8705,  0.5961]])

In [5]:
%timeit t.a

68 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd164332c70>
├── 'a' --> tensor([[-0.7862, -0.9325, -0.3381],
│                   [-0.6127,  0.8699, -2.7746]])
└── 'x' --> <FastTreeValue 0x7fd164332ca0>
    └── 'c' --> tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                        [-0.4802, -0.7260, -0.2455, -1.1421],
                        [ 0.3915,  0.8803, -0.6810, -0.8067]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961]]),
    x: Batch(
           c: tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                      [-0.4802, -0.7260, -0.2455, -1.1421],
                      [ 0.3915,  0.8803, -0.6810, -0.8067]]),
       ),
)

In [10]:
b.a

tensor([[-0.0387,  2.1786,  1.0509],
        [ 1.2989,  0.8705,  0.5961]])

In [11]:
%timeit b.a

62.2 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0889, -1.0488, -2.0534],
               [ 0.1907, -0.0112,  0.9962]]),
    x: Batch(
           c: tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                      [-0.4802, -0.7260, -0.2455, -1.1421],
                      [ 0.3915,  0.8803, -0.6810, -0.8067]]),
       ),
)

In [13]:
%timeit b.a = new_value

607 ns ± 6.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

985 ns ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

258 µs ± 7.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

250 µs ± 6.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd0b4f688b0>
├── 'a' --> tensor([[[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]],
│           
│                   [[-0.0387,  2.1786,  1.0509],
│                    [ 1.2989,  0.8705,  0.5961]]])
└── 'x' --> <FastTreeValue 0x7fd0bbbebf70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.8 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd0bbbeba30>
├── 'a' --> tensor([[-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961],
│                   [-0.0387,  2.1786,  1.0509],
│                   [ 1.2989,  0.8705,  0.5961]])
└── 'x' --> <FastTreeValue 0x7fd0bbbfc760>
    └── 'c' --> tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                        [-0.4802, -0.7260, -0.2455, -1.1421],
                 

In [23]:
%timeit t_cat(trees)

40 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.6 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]],
       
               [[-0.0387,  2.1786,  1.0509],
                [ 1.2989,  0.8705,  0.5961]]]),
    x: Batch(
           c: tensor([[[ 1.0969, -1.3155,  0.5434, -1.3069],
                       [-0.4802, -0.7260, -0.2455, -1.1421],
                       [ 0.3915,  0.8803, -0.6810, -0.8067]],
         

In [26]:
%timeit Batch.stack(batches)

185 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961],
               [-0.0387,  2.1786,  1.0509],
               [ 1.2989,  0.8705,  0.5961]]),
    x: Batch(
           c: tensor([[ 1.0969, -1.3155,  0.5434, -1.3069],
                      [-0.4802, -0.7260, -0.2455, -1.1421],
                      [ 0.3915,  0.8803, -0.6810, -0.8067],
                      [ 1.0969, -1.3155,  0.5434, -1.3069],
                      [-0.4802, -

In [28]:
%timeit Batch.cat(batches)

170 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

500 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
